# Single Cell Decompression Training

## Import Libraries

First, we import the neccessary libraries.

In [1]:
# Import libraries
import anndata as ad
from pathlib import Path
import errno
import os
# import numpy as np
# import pandas as pd

## Helper Functions

Next, we import some helper functions including the cisi segementation training function.

In [2]:
# Helper fncs
import analysis_utils


## CISI
# Import system libraries to configure code directory as module
from os.path import dirname, abspath, join
import sys

# Find code directory relative to our directory
THIS_DIR = dirname('__file__')
CODE_DIR = abspath(join(THIS_DIR, '..', 'code'))
# Add code directory to systems paths
sys.path.append(CODE_DIR)

# Import CISI training fnc.
from train_dictionary_and_compositions import train_U_and_A

## Inputs

In the first part we specify the paths to the input files (.h5ad files created from R SpatialExperiment, TIFF files generated by the steinbock pipeline and panel metadata file) and where the outputs should be stored.

In [3]:
# Specify input paths
training_data_path = Path('/mnt/bb_dqbm_volume/data/Tonsil_th152')
spe_path = Path(os.path.join(training_data_path, 'preprocessed_data/spe.h5ad'))
tiffs_path = Path(os.path.join(training_data_path, 'steinbock/img'))
panel_path = Path(os.path.join(training_data_path, 'steinbock/panel.csv'))

# Specify output path
out_path = Path(os.path.join(training_data_path, 'training'))

# Create output directory if it doesn't exist
out_path.mkdir(parents=True, exist_ok=True)

In [5]:
# Check that input files/dictionary exist
if not analysis_utils.is_valid_file(spe_path, ['.h5ad']):
    # If file is not found, throw error
    raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT),
                            spe_path)
if not analysis_utils.is_valid_file(panel_path, ['.csv']):
    # If file is not found, throw error
    raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT),
                            panel_path)
if not analysis_utils.is_valid_directory(tiffs_path):
    # If directory is not found or doesn't contain TIFF files, throw an error
    raise Exception('Input TIFFS path {0} directory does not exist or does\
    not contain any valid TIFF files.'.format(tiffs_path)) from FileNotFoundError

Next, we read in the input files for training. For this we have the Tonsil th152 dataset consisting of 5 ROIs and we read the data in once it has been processed by steinbock into segmented single cells and once using the hot pixel corrected TIFF image files directly. From this we create one numpy array of channels x cells and one numpy array containing channels vs pixels.

In [6]:
# Read in SpatialExperiment converted to anndata by cellconverter in R
spe = ad.read_h5ad(spe_path)
print(spe)

AnnData object with n_obs × n_vars = 162624 × 43
    obs: 'sample_id', 'ObjectNumber', 'area', 'major_axis_length', 'minor_axis_length', 'eccentricity', 'width_px', 'height_px', 'ROI'
    var: 'channel', 'name', 'keep', 'ilastik', 'deepcell', 'Tube.Number', 'Metal', 'use_channel'
    uns: 'X_name'
    layers: 'exprs', 'log_exprs'


In [8]:
images = analysis_utils.anndata_from_tiff(tiffs_path, panel_path)
print(images)


AnnData object with n_obs × n_vars = 9250000 × 43
    obs: 'sample_id', 'ObjectNumber', 'ROI'
    var: 'channel', 'name', 'keep', 'ilastik', 'deepcell', 'Tube Number', 'Metal'


In [ ]:
train_U_and_A(spe, os.path.join(training_data_path, 'training/segmentation/sce'), 
              test_set=('20220520_TsH_th152_cisi1_001',))

on 2: 10 measurements
on 2: 0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
       0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
       0. 0.]
on 2: 100 [0.26762779 0.24007624 0.23704598 0.2402126  0.27110022 0.27705127
       0.30238184 0.22968897 0.23097129 0.27752373 0.24152449 0.23954979
       0.26758458 0.29587924 0.27087206 0.3046134  0.24225891 0.23176481
       0.2698174  0.23160866 0.25871393 0.2751538  0.27864732 0.24903381
       0.25372021 0.27116581 0.2434467  0.27013331 0.27855023 0.22990895
       0.27770673 0.23093038 0.23525753 0.27545248 0.29121935 0.28513829
       0.23501997 0.26209853 0.28732832 0.32327657 0.27204031 0.27289503
       0.25203314 0.26582854 0.2446016  0.23540584 0.23790434 0.28166641
       0.25546072 0.27200487]
on 2: 200 [0.26762779 0.28083517 0.32455968 0.26400206 0.27110022 0.27705127
       0.30238184 0.27179524 0.26046476 0.27752373 0.2793707  0.29383169
       0.26758458 0.29587924 0.27087

on 2: 1500 [0.32656127 0.30812738 0.32455968 0.30831614 0.31032755 0.3123156
       0.31600126 0.31389848 0.31190618 0.34800397 0.31146239 0.3247743
       0.317533   0.31597719 0.31927393 0.31414464 0.3164733  0.31336035
       0.3134026  0.31240265 0.31140127 0.34598452 0.31796387 0.30951898
       0.3327082  0.3156008  0.31264673 0.30923855 0.31550035 0.31583369
       0.31553112 0.31986565 0.31800548 0.33628382 0.3067163  0.30796247
       0.30798741 0.31702166 0.34151908 0.32327657 0.31538956 0.30750171
       0.34756701 0.3256266  0.32932444 0.31755554 0.31105576 0.32479442
       0.33182065 0.31307086]
on 2: 1600 [0.32656127 0.30812738 0.32455968 0.30831614 0.31032755 0.3123156
       0.31600126 0.31389848 0.31190618 0.34800397 0.31146239 0.3247743
       0.317533   0.31597719 0.31927393 0.31414464 0.3164733  0.31336035
       0.3134026  0.31240265 0.31140127 0.34598452 0.31796387 0.30951898
       0.3327082  0.3156008  0.31264673 0.30923855 0.31550035 0.31583369
       0.315531

on 2: /home/ubuntu/anaconda3/envs/cisi_imc_env/lib/python3.10/site-packages/scipy/spatial/distance.py:630: RuntimeWarning: invalid value encountered in double_scalars
        dist = 1.0 - uv / np.sqrt(uu * vv)


Fold 0 |████████████████████████████████████████| 4/4 [100%] in 1:32:31.5 (0.00/s) 
on 2: 10 measurements
on 2: 0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
       0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
       0. 0.]
on 2: 100 [0.23043213 0.24830687 0.27208573 0.26688332 0.26277208 0.26587926
       0.26880138 0.23677504 0.24903807 0.25221074 0.25189663 0.25452345
       0.29751973 0.23229537 0.25443582 0.27743308 0.24505038 0.24941602
       0.23415846 0.24585141 0.23066211 0.26313605 0.30585209 0.25641918
       0.2534288  0.26426639 0.24978761 0.24945592 0.27873544 0.26542273
       0.23986312 0.2480943  0.24796596 0.24175273 0.25973168 0.24055447
       0.24683248 0.23407806 0.24051224 0.26635806 0.25112267 0.28980941
       0.2514961  0.23632412 0.31806324 0.23205255 0.25526251 0.26404888
       0.26428726 0.24764701]
on 2: 200 [0.26274178 0.28129234 0.27208573 0.26688332 0.26277208 0.26587926
       0.26880138 0.27966

on 2: 1500 [0.29981286 0.30544064 0.29585342 0.31076734 0.29451723 0.3023699
       0.30527455 0.3324072  0.29246252 0.30959315 0.30661879 0.31175884
       0.29751973 0.31699814 0.29280987 0.30799296 0.29438693 0.29524799
       0.2985135  0.30607243 0.29546286 0.30024698 0.30585209 0.29401047
       0.29788854 0.30001816 0.30507045 0.30352046 0.31146918 0.29618141
       0.30569632 0.2999626  0.2991021  0.3192387  0.29529399 0.30298606
       0.30209288 0.29779732 0.29742071 0.29362974 0.33406124 0.29959859
       0.29900232 0.2962564  0.31806324 0.30628404 0.33589066 0.3254383
       0.29730002 0.29885683]
on 2: 1600 [0.29981286 0.30544064 0.29585342 0.31076734 0.29451723 0.3023699
       0.30527455 0.3324072  0.33938702 0.30959315 0.30661879 0.31175884
       0.29751973 0.31699814 0.29826556 0.30799296 0.29438693 0.29524799
       0.2985135  0.30607243 0.29546286 0.30024698 0.30585209 0.29401047
       0.29788854 0.30001816 0.30507045 0.30352046 0.31146918 0.29618141
       0.30569

on 2: /home/ubuntu/anaconda3/envs/cisi_imc_env/lib/python3.10/site-packages/scipy/spatial/distance.py:630: RuntimeWarning: invalid value encountered in double_scalars
        dist = 1.0 - uv / np.sqrt(uu * vv)


Fold 1 |████████████████████████████████████████| 4/4 [100%] in 34:05.1 (0.00/s) 
on 2: 10 measurements
on 2: 0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
       0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
       0. 0.]
on 2: 100 [0.21504525 0.19287409 0.25003399 0.21211478 0.2103737  0.25196349
       0.20190069 0.18912662 0.24030431 0.18690721 0.19022008 0.25619205
       0.19854446 0.21903957 0.1912212  0.20478852 0.18555273 0.25364542
       0.21266827 0.21062081 0.17973895 0.22021491 0.18676253 0.25808635
       0.23371944 0.22976652 0.17925548 0.18988811 0.2673127  0.21814431
       0.19718213 0.26363167 0.19599838 0.19802318 0.1908385  0.17327484
       0.20969043 0.26759998 0.23969887 0.19330134 0.19202371 0.19883612
       0.18287582 0.23340864 0.1756367  0.19802953 0.18376144 0.20862155
       0.22501431 0.22599251]
on 2: 200 [0.21504525 0.23678527 0.25003399 0.21211478 0.22591099 0.25196349
       0.22300109 0.2446780